<a href="https://colab.research.google.com/github/Trip1ewhy/Duke_xAI_Wayne/blob/main/Wayne_Finetune_Llama3_with_LLaMA_Factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune Llama-3 with LLaMA Factory

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

## Install Dependencies

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 321 (delta 76), reused 149 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (321/321), 8.98 MiB | 4.41 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 410.6/410.6 MB 211.5 MB/s eta 0:00:01

### Check GPU environment

In [3]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Update Identity Dataset

In [4]:
import json

%cd /content/LLaMA-Factory/

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/LLaMA-Factory


## Dataset Update
Add the following codes to the dataset_info.json in /data

"moral_training_data": {
    "file_name": "moral_training_data.json"
  },

And upload moral_training_data.json to /data


## Login in huggingface
Using llama3.1-8B

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Wayne_duke_xAI` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

## Fine-tune model via LLaMA Board

In [7]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2024-12-04 23:07:05.579744: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 23:07:05.597733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 23:07:05.620373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 23:07:05.627293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 23:07:05.643806: I tensorflow/core/pl

## Fine-tune model via Command Line

It takes ~30min for training.

In [ ]:
# import json

# args = dict(
#   stage="sft",                        # do supervised fine-tuning
#   do_train=True,
#   model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
#   dataset="identity,alpaca_en_demo",             # use alpaca and identity datasets
#   template="llama3",                     # use llama3 prompt template
#   finetuning_type="lora",                   # use LoRA adapters to save memory
#   lora_target="all",                     # attach LoRA adapters to all linear layers
#   output_dir="llama3_lora",                  # the path to save LoRA adapters
#   per_device_train_batch_size=2,               # the batch size
#   gradient_accumulation_steps=4,               # the gradient accumulation steps
#   lr_scheduler_type="cosine",                 # use cosine learning rate scheduler
#   logging_steps=10,                      # log every 10 steps
#   warmup_ratio=0.1,                      # use warmup scheduler
#   save_steps=1000,                      # save checkpoint every 1000 steps
#   learning_rate=5e-5,                     # the learning rate
#   num_train_epochs=3.0,                    # the epochs of training
#   max_samples=500,                      # use 500 examples in each dataset
#   max_grad_norm=1.0,                     # clip gradient norm to 1.0
#   loraplus_lr_ratio=16.0,                   # use LoRA+ algorithm with lambda=16.0
#   fp16=True,                         # use float16 mixed precision training
#   use_liger_kernel=True,                   # use liger kernel for efficient training
# )

# json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

# %cd /content/LLaMA-Factory/

# !llamafactory-cli train train_llama3.json

/content/LLaMA-Factory


## Infer the fine-tuned model

In [ ]:
# from llamafactory.chat import ChatModel
# from llamafactory.extras.misc import torch_gc

# %cd /content/LLaMA-Factory/

# args = dict(
#   model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
#   adapter_name_or_path="llama3_lora",            # load the saved LoRA adapters
#   template="llama3",                     # same to the one in training
#   finetuning_type="lora",                  # same to the one in training
#   quantization_bit=4,                    # load 4-bit quantized model
# )
# chat_model = ChatModel(args)

# messages = []
# print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
# while True:
#   query = input("\nUser: ")
#   if query.strip() == "exit":
#     break
#   if query.strip() == "clear":
#     messages = []
#     torch_gc()
#     print("History has been removed.")
#     continue

#   messages.append({"role": "user", "content": query})
#   print("Assistant: ", end="", flush=True)

#   response = ""
#   for new_text in chat_model.stream_chat(messages):
#     print(new_text, end="", flush=True)
#     response += new_text
#   print()
#   messages.append({"role": "assistant", "content": response})

# torch_gc()

## Merge the LoRA adapter and optionally upload model

NOTE: the Colab free version has merely 12GB RAM, where merging LoRA of a 8B model needs at least 18GB RAM, thus you **cannot** perform it in the free version.

In [ ]:
# import json

# args = dict(
#   model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
#   adapter_name_or_path="llama3_lora",            # load the saved LoRA adapters
#   template="llama3",                     # same to the one in training
#   finetuning_type="lora",                  # same to the one in training
#   export_dir="llama3_lora_merged",              # the path to save the merged model
#   export_size=2,                       # the file shard size (in GB) of the merged model
#   export_device="cpu",                    # the device used in export, can be chosen from `cpu` and `cuda`
#   #export_hub_model_id="your_id/your_model",         # the Hugging Face hub ID to upload model
# )

# json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

# %cd /content/LLaMA-Factory/

# !llamafactory-cli export merge_llama3.json

In [ ]:
!pip install pyngrok